In [191]:
def data(cls):
    def _repr(self):
        return str(self.__dict__)

    def _hash(self):
        return hash(repr(self))

    def _eq(self, other):
        if not isinstance(other, type(self)):
            return False
        for key in self.__dict__:
            if key not in other.__dict__:
                return False
            if self.__dict__[key] != other.__dict__[key]:
                return False
        return True

    setattr(cls, '__repr__', _repr)
    setattr(cls, '__hash__', _hash)
    setattr(cls, '__eq__', _eq)
    return cls


In [192]:
@data
class Input:
  def __init__(self, fileName):
    with open(fileName, "r") as file:
      lines = file.readlines()
      self.algorithm = lines[0]
      self.image = tuple(map(lambda line: line.strip('\n'), filter(lambda line: len(line) != 0, lines[2:])))

In [193]:
def enhancePixel(image, algorithm, pixelPosition, outOfRangePixelValue):
  def getConsideredPositions():
    for x in range(-1, 2):
      for y in range(-1, 2):
        yield (pixelPosition[0] + x, pixelPosition[1] + y)

  def getPixelValue(position):
    if position[0] >= 0 and position[0] < len(image)\
      and position[1] >= 0 and position[1] < len (image[position[0]]):
      return image[position[0]][position[1]]
    else:
      return outOfRangePixelValue

  consideredPixels = ''.join(map(getPixelValue, getConsideredPositions()))
  number = int(consideredPixels.replace('.', '0').replace('#', '1'), 2)
  return algorithm[number]

In [194]:
def enhanceImage(image, algorithm, outOfRangePixelValue):
  enhanced = []
  for x in range(-1, len(image) + 1):
    row = ''
    for y in range(-1, len(image[0]) + 1):
      row += enhancePixel(image, algorithm, (x, y), outOfRangePixelValue)
    enhanced.append(row)
  return tuple(enhanced)

In [195]:
def enhanceImageMultipleTimes(image, algorithm, times):
  outOfRangePixelValue = '.'
  enhanced = image
  for _ in range(times):
    enhanced = enhanceImage(enhanced, algorithm, outOfRangePixelValue)

    # Out of range pixels will turn bright on this enhance
    if outOfRangePixelValue == '.' and algorithm[0] == '#':
      outOfRangePixelValue = '#'

    # Out of range pixels will turn dark on this enhance
    elif outOfRangePixelValue == '#' and algorithm[511] == '.':
      outOfRangePixelValue = '.'

  return enhanced

In [196]:
input = Input('input.txt')
enhanced = enhanceImageMultipleTimes(input.image, input.algorithm, 50)
print(''.join(enhanced).count('#'))


5301
